In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import sys
import numpy as np
main_dir = 'C:/Users/nourisa/Downloads/testProjs/geneRNI/'

In [10]:
# data processing and reformatting
from geneRNI import tools
from geneRNI import geneRNI
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
import importlib
importlib.reload(tools)
TS_data, time_points, SS_data, gene_names = tools.Benchmark.f_data_dream4(10,1)
Xs, ys = tools.Data.process(TS_data = TS_data, SS_data = SS_data, time_points = time_points, gene_names=gene_names, regulators = 'all', KO = None, verbose=True)

Xs = Xs[0] #because they are all same

ys = np.array(ys) 
ys = ys.T # to change the sample and gene order
print(Xs.shape)
print(ys.shape)

X_train, X_test, y_train, y_test = model_selection.train_test_split(Xs, ys, test_size=.2)

dynamic data: ngenes: 10, nsamples: 90, n regulators: 10
static data: ngenes: 10, nsamples: 31, n regulators: 10
(121, 10)
(121, 10)


In [3]:
# regressor 
from sklearn import base
from sklearn import utils
from sklearn import ensemble
class GenesEstimator(base.BaseEstimator,base.RegressorMixin):
    """The docstring for a class should summarize its behavior and list the public methods and instance variables """
    def __init__(self, decay_coeff = 0, **params):
        '''args should all be keyword arguments with a default value -> kwargs should be all the keyword params of all regressors with values'''
        '''they should not be documented under the “Attributes” section, but rather under the “Parameters” section for that estimator.'''
        '''every keyword argument accepted by __init__ should correspond to an attribute on the instance'''
        '''There should be no logic, not even input validation, and the parameters should not be changed. The corresponding logic should be put where the parameters are used, typically in fit'''
        '''algorithm-specific unit tests,'''
        # self.decay_coeff = decay_coeff
        self.params = params
        self.decay_coeff = decay_coeff
        self.est = None
        # self._required_parameters = () #estimators also need to declare any non-optional parameters to __init__ in the
    def fit(self, Xs, ys):
        """ fit X to y
        X -- Array-like of shape (n_samples, n_features)
        y -- Array-like of shape (n_samples,)
        kwargs -- Optional data-dependent parameters
        """
        '''Attributes that have been estimated from the data must always have a name ending with trailing underscore'''
        '''The estimated attributes are expected to be overridden when you call fit a second time.'''
        
        # apply decay_coeff to y
        ys = np.array([[y_i(self.decay_coeff) for y_i in y] for y in ys])
#         utils.check_array(Xs)
#         utils.check_X_y(Xs,ys)
#         utils.indexable(Xs)
#         utils.indexable(ys)

        self.X_ = Xs
        self.y_ = ys
        
        self.est = ensemble.RandomForestRegressor(oob_score = True,**self.params)
        
        self.est.fit(Xs,ys)
        return self
    def predict(self,Xs):
        """ """
        utils.validation.check_is_fitted(self.est)
        return self.est.predict(Xs)
    def score(self, Xs, ys): 
        """ """
        # apply decay_coeff to y
        ys = np.array([[y_i(self.decay_coeff) for y_i in y] for y in ys])
        utils.validation.check_is_fitted(self.est)
#         utils.check_array(Xs)
#         utils.check_X_y(Xs,ys)
#         utils.indexable(Xs)
#         utils.indexable(ys)
        # print(self.est.score(X,y))
        return self.est.score(Xs,ys)

    
    def get_params(self,deep=True):
        """ The get_params function takes no arguments and returns a dict of 
        the __init__ parameters of the estimator, together with their values. 

        """
        return { 'decay_coeff': self.decay_coeff, **self.params}
    def set_params(self, **parameters):
        """ """
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    def _more_tags(self):
        """ """
        allow_nan = False
        return {'requires_fit': True, 'allow_nan': allow_nan, 'multioutput': True, 
            'requires_y': True,}

In [4]:
# reg = GenesEstimator()
# reg.fit(X_train, y_train)
# reg.score(X_test, y_test)

0.5217633015384048

In [5]:
# param tuning sklearn
# from sklearn import model_selection
# distributions = dict(decay_coeff=np.arange(0,1,.1))
# clf = model_selection.RandomizedSearchCV(reg, param_distributions=distributions, n_iter=10, n_jobs=10, cv=4, random_state=None)
# outs = clf.fit(X_train, y_train) 

In [20]:
# param tuning 
from geneRNI import search_param_mv
import importlib
importlib.reload(search_param_mv)
distributions = dict(decay_coeff=np.arange(0,1,.1))
best_score, best_param, sampled_permts_sorted = search_param_mv.rand_search(reg, X_train, y_train, param_grid=distributions, n_jobs = 1)

stats: 10 permts 1 threads
Running 10 samples randomly
Param search is completed in 1.060 seconds


In [21]:
print(best_score)


0.6575148473423764
